### listの動作

In [1]:
layers = [1,2,3]
layers += [5,6,7]
print(layers)

[1, 2, 3, 5, 6, 7]


### torch.Tensorの次元の増減

In [2]:
import torch
from torch import nn
from torch.nn import init

In [3]:
weight = nn.Parameter(torch.Tensor(512))
init.constant_(weight, 20)
print(weight.size())

torch.Size([512])


In [4]:
weight.unsqueeze(0).size()

torch.Size([1, 512])

In [5]:
weight.unsqueeze(0).unsqueeze(2).size()

torch.Size([1, 512, 1])

In [6]:
weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).size()

torch.Size([1, 512, 1, 1])

In [7]:
batch_size = 5
x = torch.ones((batch_size, 512, 38, 38))
x.size()

torch.Size([5, 512, 38, 38])

### 対象の変数の次元を別の変数の次元と同じにする

In [8]:
ww = weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)
print(ww.size())

torch.Size([5, 512, 38, 38])


In [9]:
x.pow(2).size()

torch.Size([5, 512, 38, 38])

In [14]:
x.pow(2).sum(dim=1).size()

torch.Size([5, 38, 38])

### 次元を保ったままの和算

In [11]:
x.pow(2).sum(dim=1, keepdim=True).size()

torch.Size([5, 1, 38, 38])

In [12]:
xx = x.pow(2).sum(dim=1, keepdim=True).sqrt()+1e-10
print(xx.size())
print(xx[0, :, :, :])

torch.Size([5, 1, 38, 38])
tensor([[[22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         ...,
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274]]])


In [13]:
wx = xx * ww
print(wx.size())
print(wx)

torch.Size([5, 512, 38, 38])
tensor([[[[452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          ...,
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483]],

         [[452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          ...,
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483]],

 

### list or np.ndarray からtorch.tensorを作る

In [15]:
torch.tensor([[1.0, -1.0], [1.0, -1.0]])

tensor([[ 1., -1.],
        [ 1., -1.]])

In [17]:
import numpy as np
torch.tensor(np.array([[1,2,3],[4,5,6]]))

tensor([[1, 2, 3],
        [4, 5, 6]])

torch.tensor()はいつもデータをコピーする

### データ・タイプを指定してtensorを作る

In [18]:
torch.zeros([2,4], dtype=torch.int32)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0]], dtype=torch.int32)

### torch.tensorのスライシング

In [20]:
x = torch.tensor([[1,2,3],[4,5,6]])
print(x[0][1])

tensor(2)


In [21]:
x[0][1] = 8
print(x)

tensor([[1, 8, 3],
        [4, 5, 6]])


### torch.tensorからpython定義の数値を抽出する

In [22]:
x = torch.tensor([[1]])
print(x)
print(x.item())

tensor([[1]])
1


In [28]:
x = torch.tensor(2.5)
print(x)
print(x.item())
print(x.layout)

tensor(2.5000)
2.5
torch.strided


### torch.tensorは自動微分OFFの状態で生成される

In [29]:
x = torch.tensor([[1., -1.], [1., 1.]])
print("requires_grad: ", x.requires_grad)
print(x.layout)

requires_grad:  False
torch.strided


In [27]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
print("requires_grad: ", x.requires_grad)
print("x.grad: ", x.grad)
out = x.pow(2).sum()
print("out: ", out)
out.backward()
print("x.grad after out.backward(): \n", x.grad)

requires_grad:  True
x.grad:  None
out:  tensor(4., grad_fn=<SumBackward0>)
x.grad after out.backward(): 
 tensor([[ 2., -2.],
        [ 2.,  2.]])


現在の実装では, torch.tensorはメモリのオーバヘッドが大きいので、小さな複数のtorch.tensorを運用するのはメモリ消費が激しくなる.

### dtypeとdeviceを引き継きながら、既存データからデータをコピーしたtorch.tensorを作る

In [38]:
tensor = torch.ones((2,), dtype=torch.int8)
data = [[0,1],[2,3]]
print(tensor.new_tensor(data))
print(tensor.clone())
print(tensor.clone().detach())

tensor([[0, 1],
        [2, 3]], dtype=torch.int8)
tensor([1, 1], dtype=torch.int8)
tensor([1, 1], dtype=torch.int8)


### requires_gradを変更する

In [34]:
x = torch.tensor([[0,1],[2,3]], dtype=torch.float32)
print(x.requires_grad)
x.requires_grad_(True) # 浮動小数点型のみ勾配変数を持つ
print(x.requires_grad)

False
True


new_full(size,fill_value,dtype=None,device=None,requires_grad=False) -> Tensor

In [44]:
# dtypeとdeviceを継承
tensor = torch.ones((2,), dtype=torch.float64)
tensor.new_full((3,4), fill_value=3.141592)

tensor([[3.1416, 3.1416, 3.1416, 3.1416],
        [3.1416, 3.1416, 3.1416, 3.1416],
        [3.1416, 3.1416, 3.1416, 3.1416]], dtype=torch.float64)

new_empty(size,dtype=None,device=None,requires_grad=False) -> Tensor

In [47]:
tensor = torch.ones(())
tensor = tensor.new_empty((2,3))
print(tensor)
print(tensor.dtype)

tensor([[1.1210e-44, 0.0000e+00, 7.0373e-25],
        [1.4013e-45, 0.0000e+00, 0.0000e+00]])
torch.float32


new_ones(size,dtype=None,device=None,requires_grad=False) -> Tensor

In [50]:
tensor = torch.tensor((), dtype=torch.int32)
tensor.new_ones((2,3))

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int32)

new_zero(size,dtype=None,device=None,requires_grad=False) -> Tensor

In [51]:
tensor = torch.tensor((), dtype=torch.float64)
tensor.new_zeros((2,3))

tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

### torch.tensorがGPUメモリにあるか否かのチェック

In [53]:
tensor = torch.tensor((), dtype=torch.float32)
print(tensor)
print(tensor.is_cuda)

tensor([])
False


### quantizedか否かのチェック

In [54]:
print(tensor.is_quantized)

False


### tensorのdeviceチェック

In [55]:
print(tensor.device)

cpu


### 次元

In [56]:
tensor.dim()

1

### 転置

In [58]:
x = torch.tensor([[0,1],[2,3]], dtype=torch.int32)
print(x)
print(x.T)

tensor([[0, 1],
        [2, 3]], dtype=torch.int32)
tensor([[0, 2],
        [1, 3]], dtype=torch.int32)


### 要素数を維持しながらtensorの形状を変える。もし引数に与えた要素数が元の数値より低い場合は、元のtensorの中身は削られる

In [89]:
a = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
y = a.clone().detach()
x = torch.tensor([0,1,2,3,4,5,6,7,8])
print("a shape", a.size())
print("x shape", x.size())
print("resized a shape", a.resize_as_(x))

c = a.clone().detach() # 元tensor:aの自動微分の関係を引き継がないためにdetach()が必要。
print("c", c)
print(c is a)

b = torch.tensor([[9,8],[6,5]])
print("b shape", b.size())
print("resized a shape", a.resize_as_(b))
print("a shape", a.size())

e = torch.tensor([[0,0],[0,0]])
print("y", y)
print("resized y", y.resize_as_(e))

a shape torch.Size([3, 3])
x shape torch.Size([9])
resized a shape tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
c tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
False
b shape torch.Size([2, 2])
resized a shape tensor([[0, 1],
        [2, 3]])
a shape torch.Size([2, 2])
y tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])


RuntimeError: set_sizes_contiguous is not allowed on a Tensor created from .data or .detach().
If your intent is to change the metadata of a Tensor (such as sizes / strides / storage / storage_offset)
without autograd tracking the change, remove the .data / .detach() call and wrap the change in a `with torch.no_grad():` block.
For example, change:
    x.data.set_(y)
to:
    with torch.no_grad():
        x.set_(y)

In [92]:
num_batch = 5
conf_t_label = torch.randint(0, 21,(num_batch, 8732))
print(conf_t_label.size())
print(conf_t_label)

torch.Size([5, 8732])
tensor([[ 2, 17,  4,  ..., 12,  3, 13],
        [ 1, 13, 15,  ...,  7, 20, 12],
        [15, 13, 20,  ...,  6, 15, 18],
        [ 9,  9,  7,  ...,  8, 16, 14],
        [ 0,  3,  4,  ...,  0,  8,  4]])


In [93]:
pos_mask = conf_t_label > 0
print(pos_mask)

tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [False,  True,  True,  ..., False,  True,  True]])


In [94]:
num_pos = pos_mask.long().sum(dim=1, keepdim=True)
print(num_pos)

tensor([[8303],
        [8337],
        [8304],
        [8314],
        [8327]])


In [95]:
loss_c = torch.randn((num_batch * 8732,))
print(loss_c.size())
print(loss_c)

torch.Size([43660])
tensor([-0.6359, -0.8697,  0.5686,  ..., -1.2054, -1.1376,  1.0267])


In [96]:
loss_c2 = loss_c.view(num_batch, -1)
print(loss_c2.size())
print(loss_c2)

torch.Size([5, 8732])
tensor([[-0.6359, -0.8697,  0.5686,  ...,  0.8119, -3.1378, -1.0195],
        [ 1.3421,  1.1616,  1.1381,  ..., -0.5337,  0.2220, -1.9861],
        [-0.1525,  0.7679,  1.5779,  ..., -1.2768,  0.5376,  0.7597],
        [ 1.6534, -0.3884, -0.3403,  ...,  1.2776, -1.5996,  0.0105],
        [ 0.5576, -1.5604, -0.5169,  ..., -1.2054, -1.1376,  1.0267]])


In [99]:
print(loss_c2[pos_mask])
print(loss_c2[pos_mask].size())

tensor([-0.6359, -0.8697,  0.5686,  ...,  0.3195, -1.1376,  1.0267])
torch.Size([41585])


In [101]:
loss_c3 = loss_c2.clone()
loss_c3[pos_mask] = 0
print(loss_c3.size())
print(loss_c3)

torch.Size([5, 8732])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.5576,  0.0000,  0.0000,  ..., -1.2054,  0.0000,  0.0000]])


In [102]:
sorted_loss_c3, mapping_loss_c3_index = loss_c3.sort(dim=1, descending=True)

In [103]:
print(sorted_loss_c3)

tensor([[ 2.7124,  2.5287,  2.3202,  ..., -2.9559, -3.1229, -3.2518],
        [ 2.7407,  2.6826,  2.5023,  ..., -2.3404, -2.6146, -2.7753],
        [ 3.4014,  2.6461,  2.5337,  ..., -2.6235, -2.6949, -3.1518],
        [ 2.5460,  2.4067,  2.2981,  ..., -2.5410, -2.8131, -3.5553],
        [ 2.6428,  2.5604,  2.5286,  ..., -1.9720, -1.9876, -2.0845]])


In [104]:
print(mapping_loss_c3_index)

tensor([[4211, 2647, 5002,  ..., 5506,  968,   91],
        [8639, 4187, 4762,  ..., 6760, 2741, 7575],
        [3664, 5108, 6315,  ..., 2786, 2742, 1756],
        [2957,  601,   62,  ..., 7450, 7385, 5642],
        [4187, 8213, 7061,  ..., 2539, 1140, 2984]])


In [107]:
sorted_mapping, loss_rank = mapping_loss_c3_index.sort(dim=1)
print(sorted_mapping)
print(loss_rank)

tensor([[   0,    1,    2,  ..., 8729, 8730, 8731],
        [   0,    1,    2,  ..., 8729, 8730, 8731],
        [   0,    1,    2,  ..., 8729, 8730, 8731],
        [   0,    1,    2,  ..., 8729, 8730, 8731],
        [   0,    1,    2,  ..., 8729, 8730, 8731]])
tensor([[4368, 7212, 5788,  ..., 2952, 2953, 7962],
        [4368, 7213, 5789,  ..., 2947, 2948, 8295],
        [4367, 7194, 5778,  ..., 2947, 2948, 7541],
        [4370, 7210, 5789,  ..., 2949, 2950, 7566],
        [ 112, 4365, 5782,  ..., 8686, 2947, 7293]])


In [108]:
print(loss_c3[0, loss_rank[0, 0]])

tensor(0.)


In [109]:
print(loss_c3[0, loss_rank[0, 1]])

tensor(0.)
